## Assignment 3: $k$ Nearest Neighbor and $k$ Means Clustering

## **Do three questions.**

`! git clone https://www.github.com/DS3001/assignment3`

In [ ]:
! git clone https://www.github.com/DS3001/assignment3

Cloning into 'assignment3'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 14 (delta 2), reused 1 (delta 1), pack-reused 7
Receiving objects: 100% (14/14), 868.21 KiB | 5.95 MiB/s, done.
Resolving deltas: 100% (2/2), done.


**Q1.** This question is a case study for $k$ nearest neighbor. The target variable `y` is `Purchase` and the features of interest are `Age` and `AnnualSalary`.

1. Load the `./data/car_data.csv` data. Look at the head and dimensions of the data.
2. Summarize the variables (`User.ID`, `Gender`, `Age`, `AnnualSalary`, `Purchase`). Are there any missings to clean? Convert the `Gender` categorical variable into a dummy variable that takes the value 0 for male and 1 for female. Create a matrix $X$ of predictors including `Age` and `AnnualSalary`, and an outcome $y$ equaling `Purchase`.
3. MaxMin-normalize `Age` and `AnnualSalary` in `X`.
4. Split the sample into a ~80% training dataset and a ~20% testing dataset.
5. Treat this as a classification problem: The model is supposed to predict 0 or 1 for each customer, classifying them as a purchaser or non-purchaser. Use sklearn to determine the optimal number of neighbors $k$ to use.
6. Run the model for the optimal number of neighbors on the testing data. Cross tabulate the predicted outcomes against the actual outcomes; this is called a **confusion matrix**. How often does the model predict a sale when one fails to occur? How often does the model predict no sale when one does occur? Overall, does it provide accurate predictions?
7. Now, compute confusion matrices separately for men and women, as in part 6. Does the model make more accurate predictions for one sex or the other? Explain. (Performance of algorithms on population subgroups is a growing topic in data science.)

In [ ]:
#1
import pandas as pd
import numpy as np
car = pd.read_csv('./assignment3/data/car_data.csv')
print(car.columns)
car.head()

#2 Matrix
car.describe()
car['gdummy'] = 0
car.loc[ car['Gender']== 'Female', 'gdummy'] = 1
car['gdummy'].value_counts()

matrix = ['AnnualSalary', 'Age']
X = car.loc[:,matrix]
Y = car['Purchased']

#3 MaxMin-normalize
def maxmin(x):
  x = (x-min(x))/(max(x)-min(x))
  return x
X = X.apply(maxmin)
X.describe()

#4 Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.2,random_state=1)

#5 Optimal k (reference from provided solution bc i got really stumped on this one)
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

k_bar = 30
k_grid = np.arange(1,k_bar)
SSE = np.zeros(k_bar)
for k in range(k_bar):
  model = KNeighborsClassifier(n_neighbors=k+1)
  fitted_model = model.fit(X_train,y_train)
  y_hat = fitted_model.predict(X_test)
  SSE[k] = np.sum( (y_test-y_hat)**2 )
SSE_min = np.min(SSE)
min_index = np.where(SSE==SSE_min)
k_star = k_grid[min_index]
print(k_star)

plt.plot(np.arange(0,k_bar),SSE,label='Test')
plt.xlabel("k")
plt.ylabel("SSE")
plt.legend(loc='lower right')
plt.title('SSE- k neighbors')
plt.show()

#6 Confusion Matrix
confusion = KNeighborsClassifier(n_neighbors=9)
conmat = confusion.fit(X_train,y_train)
y_hat = conmat.predict(X_test)
pd.crosstab(y_test, y_hat)
# when a sale is not made, the model accurately predicts it 114 times, but when a sale is made but the model predicts that one wasn't, the model predicts this inaccurately 22 times
# when a sale is made, the model accurately predicts it 53 times, when a sale is not made but the model predicts that one was, it happens 11 times

#7 Confusion Matrix by Sex
from sklearn.model_selection import train_test_split
bygender = ['Gender','Age','AnnualSalary']
X = car.loc[:,bygender]
y = car['Purchased']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1)

genmat = KNeighborsClassifier(n_neighbors=9)
genmat_con = genmat.fit(X_train.drop('Gender',axis=1),y_train)
y_hat = genmat_con.predict(X_test.drop('Gender',axis=1))

My_hat = y_hat[ X_test['Gender'] == 'Male']
Fy_hat = y_hat[ X_test['Gender'] == 'Female']
My = y_test[ X_test['Gender'] == 'Male']
Fy = y_test[ X_test['Gender'] == 'Female']

pd.crosstab(Fy, Fy_hat)
# the model predicts accurately for women about .76 times (80/105)
pd.crosstab(My, My_hat)
# the model predicts accurately for men about .83 times (81/97) showing that there is a higher accuracy rate for men over women just comparing the rates alone
# calculating the difference between the accuracy rates by gender (.83-.76)/.6796 = .1084) there is about an 11% difference between gender, which is kind of a high differential to be had on the basis of gender alone

**Q2.** This question is a case study for $k$ nearest neighbor The target variable `y` is `price` and the features are `year` and `mileage`.

1. Load the `./data/USA_cars_datasets.csv`. Keep the following variables and drop the rest: `price`, `year`, `mileage`. Are there any `NA`'s to handle? Look at the head and dimensions of the data.
2. Maxmin normalize `year` and `mileage`.
3. Split the sample into ~80% for training and ~20% for evaluation.
4. Use the $k$NN algorithm for regression and the training data to predict `price` using `year` and `mileage` for the test set for $k=3,10,25,50,100,300$. For each value of $k$, compute the Sum of Squared Error and make a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase $k$?
5. Determine the optimal $k$ for these data.
6. Describe what happened in the plots of predicted versus actual prices as $k$ varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor as kNNRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

*Question 2 Part 1*

In [ ]:
usa = pd.read_csv('/content/assignment3/data/USA_cars_datasets.csv')
usa_cars = usa.drop(['brand', 'model', 'title_status', 'color', 'vin', 'lot', 'state', 'country', 'condition'], axis=1)
usa_cars.isnull().values.any() #no NA values bc it runs false
usa_cars.head()

*Question 2 Part 2 (Maxmin)*

In [ ]:
y = usa_cars['price']
x = usa_cars.loc[:,['year','mileage']]

def maxmin(x):
    x = (x-min(x))/(max(x)-min(x))
    return(x)

X = x.apply(maxmin)
X.describe()

*Question 2 Part 3 (split)*

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=1)

*Question 2 Part 4 (regression)*

In [ ]:
# K-Value 3
# as the K value increases, sse decreases

k = 3 #set k-value
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test) #predicting price
sse = mean_squared_error(y_test, y_hat) * len(y_test) #calculating sse

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')


In [ ]:
# K-Value 10

k = 10
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test)
sse = mean_squared_error(y_test, y_hat) * len(y_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')

In [ ]:
# K-Value 25
k = 25
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test)
sse = mean_squared_error(y_test, y_hat) * len(y_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')

In [ ]:
# K-Value 50
k = 50
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test)
sse = mean_squared_error(y_test, y_hat) * len(y_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')

In [ ]:
# K-Value 100
k = 100
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test)
sse = mean_squared_error(y_test, y_hat) * len(y_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')

In [ ]:
# K-Value 300
k = 300
model = kNNRegression(n_neighbors=k)
fitted_model = model.fit(X_train, y_train)

y_hat = fitted_model.predict(X_test)
sse = mean_squared_error(y_test, y_hat) * len(y_test)

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_hat)
plt.title(f'Scatterplot for k={k}')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)

plt.show()
print(f'k={k}, SSE={sse}')

*Question 2 Part 5*

In [ ]:
k_bar = 70
k_grid = np.arange(1,k_bar)
k_star = k_grid[min_index]
print('Optimal K-value:',k_star)

#optimal k-value is 3

Optimal K-value: [3]


*Question 2 Part 6*
- When k=300, the graph had the potential to underfit the data and portraying an accurate SSE. Because 300 is such a large value, it has the tendency to oversimplify data, assuming a greater amount of data per value and thus, failing to account for any "nuances"
- when k=3, the graph was able to show a lot of variability as the amount of "neighbors" considered was not a large group.

**Q6.** This is a question about $k$ means clustering. We want to investigate how adjusting the "noisiness" of the data impacts the quality of the algorithm and the difficulty of picking $k$.

1. Run the code below, which creates four datasets: `df0_125`, `df0_25`, `df0_5`, `df1_0`, and `df2_0`. Each data set is created by increasing the amount of `noise` (standard deviation) around the cluster centers, from `0.125` to `0.25` to `0.5` to `1.0` to `2.0`.

```
import numpy as np
import pandas as pd

def createData(noise,N=50):
    np.random.seed(100) # Set the seed for replicability
    # Generate (x1,x2,g) triples:
    X1 = np.array([np.random.normal(1,noise,N),np.random.normal(1,noise,N)])
    X2 = np.array([np.random.normal(3,noise,N),np.random.normal(2,noise,N)])
    X3 = np.array([np.random.normal(5,noise,N),np.random.normal(3,noise,N)])
    # Concatenate into one data frame
    gdf1 = pd.DataFrame({'x1':X1[0,:],'x2':X1[1,:],'group':'a'})
    gdf2 = pd.DataFrame({'x1':X2[0,:],'x2':X2[1,:],'group':'b'})
    gdf3 = pd.DataFrame({'x1':X3[0,:],'x2':X3[1,:],'group':'c'})
    df = pd.concat([gdf1,gdf2,gdf3],axis=0)
    return df

df0_125 = createData(0.125)
df0_25 = createData(0.25)
df0_5 = createData(0.5)
df1_0 = createData(1.0)
df2_0 = createData(2.0)
```

2. Make scatterplots of the $(X1,X2)$ points by group for each of the datasets. As the `noise` goes up from 0.125 to 2.0, what happens to the visual distinctness of the clusters?
3. Create a scree plot for each of the datasets. Describe how the level of `noise` affects the scree plot (particularly the presence of a clear "elbow") and your ability to definitively select a $k$.
4. Explain the intuition of the elbow, using this numerical simulation as an example.

*Question 6 Part 1*

In [ ]:
# Question 6 Part 1
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def createData(noise,N=50):
    np.random.seed(100)

    trip_X1 = np.array([np.random.normal(1,noise,N),np.random.normal(1,noise,N)])
    trip_X2 = np.array([np.random.normal(3,noise,N),np.random.normal(2,noise,N)])
    trip_X3 = np.array([np.random.normal(5,noise,N),np.random.normal(3,noise,N)])

    con_1 = pd.DataFrame({'x1':trip_X1[0,:],'x2':trip_X1[1,:],'group':'a'})
    con_2 = pd.DataFrame({'x1':trip_X2[0,:],'x2':trip_X2[1,:],'group':'b'})
    con_3 = pd.DataFrame({'x1':trip_X3[0,:],'x2':trip_X3[1,:],'group':'c'})
    con_whole = pd.concat([con_1,con_2,con_3],axis=0)
    return con_whole

cw0_125 = createData(0.125)
cw0_25 = createData(0.25)
cw0_5 = createData(0.5)
cw1_0 = createData(1.0)
cw2_0 = createData(2.0)

*Question 6 Part 2*

In [ ]:
# Question 6 Part 2
# As the noise increases by increments, the visual distinctness of the clusters begin to get more dispersed and overlap.
# 0.125 Scatterplot
sns.scatterplot(data = cw0_125, x = 'x1',y='x2',hue='group',style='group')

In [ ]:
# 0.25 Scatterplot
sns.scatterplot(data = cw0_25, x = 'x1',y='x2',hue='group',style='group')

In [ ]:
# 0.5 Scatterplot
sns.scatterplot(data = cw0_5, x = 'x1',y='x2',hue='group',style='group')

In [ ]:
# 1.0 Scatterplot
sns.scatterplot(data = cw1_0, x = 'x1',y='x2',hue='group',style='group')

In [ ]:
# 2.0 Scatterplot
sns.scatterplot(data = cw2_0, x = 'x1',y='x2',hue='group',style='group')

*Question 6 Part 3*

In [ ]:
# QUESTION 6 PART 3 (reference also from provided solution- giving credit where it's due!)
def maxmin(x):
  x = (x-min(x))/(max(x)-min(x))
  return x

def scree(data):
  X = data.loc[ : , ['x1','x2'] ]
  X = X.apply(maxmin)

  k_bar = 15
  k_grid = np.arange(1,k_bar+1)
  SSE = np.zeros(k_bar)
  for k in range(k_bar):
    model = KMeans(n_clusters=k+1, max_iter=300, n_init = 10, random_state=0)
    model = model.fit(X)
    SSE[k] = model.inertia_
  scree_plot, axes = plt.subplots()
  sns.lineplot(x=k_grid, y=SSE).set_title('Scree Plot')
  axes.set_ylim(0, 35)

In [ ]:
# 0.125 Scree Plot
scree( data = cw0_125 )

In [ ]:
# 0.25 Scree Plot
scree( data = cw0_25)

In [ ]:
# 0.5 Scree Plot
scree( data = cw0_5)
# For these first three plots, there is a very clear elbow presenting at k=3

In [ ]:
# 1.0 Scree Plot
scree( data = cw1_0)

In [ ]:
# 2.0 Scree Plot
scree( data = cw2_0)
# For these final two plots, there isn't really a clear elbow present? So, as the noise increases, there becomes a lack of clarity.

Drawing from the noise plots done in part 2, it became clear that as the noise increases by increments, the visual distinctness of the clusters begin to get more dispersed and overlap. Correlating this information with the fact that "elbows" were only present in the first three plots where there was less noise, a direct relationship occurs between noise and the plausbility of scree plots accurately displaying: as noise increases, it becomes more difficult to discern k-levels.